In [6]:
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt


In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
df=pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [11]:
df.isnull().sum()


age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [41]:
df['city'].value_counts()


city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop('has_covid',axis=1),
                                               df['has_covid'],
                                               test_size=0.2,
                                               random_state=0)
X_train.shape,X_test.shape,y_train.shape

((80, 5), (20, 5), (80,))

In [20]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


## without column transform


In [62]:
#filling missing value using imputer class
si=SimpleImputer()
# performance fit and transform together
X_train_fever=si.fit_transform(X_train[['fever']])
# performance fit on also test data for btter comparison
X_test_fever=si.fit_transform(X_test[['fever']])
X_train_fever.shape,X_test_fever.shape


((80, 1), (20, 1))

In [63]:
#Ordinal encoding on cough
oe=OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=oe.fit_transform(X_train[['cough']])
X_test_cough=oe.fit_transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [64]:
#One Hot Encoding -> gender city 
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city=ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city=ohe.fit_transform(X_test[['gender','city']])
# 4 column because we drop 1-1 from both columns
X_train_gender_city.shape



(80, 4)

In [65]:
#Extracting age
X_train_age=X_train.drop(columns=['gender','fever','city','cough']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape,X_test_age.shape



((80, 1), (20, 1))

In [69]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)
X_train_transformed.shape,X_test_transformed.shape

((80, 7), (20, 7))

## without column transformed

In [72]:
from sklearn.compose import ColumnTransformer
transformer=ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first',sparse_output=False),['gender','city'])],
    remainder='passthrough')

In [73]:
transformer.fit_transform(X_train).shape

(80, 7)

In [74]:
transformer.fit_transform(X_test).shape

(20, 7)